**Задание:**
Используя данные из обучающего датасета (train.csv), построить модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели, предсказать цены для квартир из тестового датасета (test.csv).

**Целевая переменная:**
Price

**Метрика качества:**
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

**Требования к решению:**
1. R2 > 0.6
2. Тетрадка Jupyter Notebook с кодом Вашего решения, названная по образцу {ФИО}_solution.ipynb, пример SShirkin_solution.ipynb
3. Файл CSV с прогнозами целевой переменной для тестового датасета, названный по образцу {ФИО}_predictions.csv, пример SShirkin_predictions.csv 
Файл должен содержать два поля: Id, Price и в файле должна быть 5001 строка (шапка + 5000 предсказаний).

**Описание датасета:**
Id - идентификационный номер квартиры
DistrictId - идентификационный номер района
Rooms - количество комнат
Square - площадь
LifeSquare - жилая площадь
KitchenSquare - площадь кухни
Floor - этаж
HouseFloor - количество этажей в доме
HouseYear - год постройки дома
Ecology_1, Ecology_2, Ecology_3 - экологические показатели местности
Social_1, Social_2, Social_3 - социальные показатели местности
Healthcare_1, Helthcare_2 - показатели местности, связанные с охраной здоровья
Shops_1, Shops_2 - показатели, связанные с наличием магазинов, торговых центров
Price - цена квартиры

In [14]:
# Запуск библиотек и скриптов
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.model_selection import KFold, GridSearchCV 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import r2_score as r2 

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
#%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [2]:
# оптимизация
def optimizing_df(df):
    for col in df.columns:
        if df[col].dtypes.kind == 'i' or df[col].dtypes.kind == 'u':
            if df[col].min() >= 0:
                df[col] = pd.to_numeric(df[col], downcast='unsigned')
            else:
                df[col] = pd.to_numeric(df[col], downcast='integer')

        elif df[col].dtypes.kind == 'f' or df[col].dtypes.kind == 'c':
            df[col] = pd.to_numeric(df[col], downcast='float')

        elif df[col].dtypes.kind == 'O':
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if num_unique_values / num_total_values < 0.5:
                df[col] = df[col].astype('category')

    return df

In [223]:
# загружаем данные 
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [103]:
#train.head()

In [104]:
#test.head()

In [105]:
#начинаем обрабатывать данные
#train.info(memory_usage='deep')

In [106]:
#test.info(memory_usage='deep')

In [224]:
# для значений которые точно должны быть целыми числами, заменим float на uint
train['HouseFloor'] = train['HouseFloor'].astype(int)
test['HouseFloor'] = test['HouseFloor'].astype(int)
train['Rooms'] = train['Rooms'].astype(int)
test['Rooms'] = test['Rooms'].astype(int)
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  int32  
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  int32  
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

In [225]:
#оптимизируем данные
train = optimizing_df(train)
test = optimizing_df(test)

In [226]:
#выбросим целевую переменную из датасета
train_target = train['Price'].astype(int)
train.drop(['Price'], axis=1, inplace=True)

In [227]:
train_target

0       184966
1       300009
2       220925
3       175616
4       150226
         ...  
9995    196684
9996    189050
9997    159143
9998    181595
9999    218714
Name: Price, Length: 10000, dtype: int32

In [228]:
# удалим признаки которые нам не понадобятся
train.drop(['Id'], axis=1, inplace=True)
train.drop(['DistrictId'], axis=1, inplace=True)
test.drop(['Id'], axis=1, inplace=True)
test.drop(['DistrictId'], axis=1, inplace=True)

In [229]:
# смотрим на призднаки и начинаем чинить их
train.describe().transpose().astype(int)

,count,mean,std,min,25%,50%,75%,max
Rooms,10000,1,0,0,1,2,2,19
Square,10000,56,21,1,41,52,65,641
LifeSquare,7887,37,86,0,22,32,45,7480
KitchenSquare,10000,6,28,0,1,6,9,2014
Floor,10000,8,5,1,4,7,12,42
HouseFloor,10000,12,6,0,9,13,17,117
HouseYear,10000,3990,200500,1910,1974,1977,2001,20052011
Ecology_1,10000,0,0,0,0,0,0,0
Social_1,10000,24,17,0,6,25,36,74
Social_2,10000,5352,4006,168,1564,5285,7227,19083


In [212]:
#test.describe().transpose().astype(int)

In [230]:
# выгрузим кол-во комнат = 0 и больше 6.
train.loc[(train['Rooms'] > 6) | (train['Rooms'] == 0)]

,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
377,10,59.056976,36.223072,10.0,22,22,2002,0.090799,B,B,74,19083,2,NaN,5,15,B
1397,0,138.427689,136.215500,0.0,4,3,2016,0.075424,B,B,11,3097,0,NaN,0,0,B
1454,19,42.006046,21.779287,7.0,17,17,2014,0.007122,B,B,1,264,0,NaN,0,1,B
1981,0,212.932358,211.231125,0.0,2,3,2008,0.211401,B,B,9,1892,0,NaN,0,1,B
2269,0,41.790882,NaN,0.0,13,0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B
3911,0,49.483501,NaN,0.0,16,0,2015,0.118537,B,B,30,6207,1,1183.0,1,0,B
4366,0,81.491447,NaN,0.0,4,0,1977,0.243205,B,B,5,1564,0,540.0,0,0,B
4853,0,2.377248,0.873147,0.0,1,0,1977,0.017647,B,B,2,469,0,NaN,0,0,B
6149,0,38.697117,19.345131,9.0,9,16,1982,0.127376,B,B,43,8429,3,NaN,3,9,B
8834,0,87.762619,85.125473,0.0,5,15,1977,0.211401,B,B,9,1892,0,NaN,0,1,B


In [232]:
# заменим на медиану
train.loc[train['Rooms'] > 6, "Rooms"] = train['Rooms'].median()
train.loc[train['Rooms'] == 0, "Rooms"] = train['Rooms'].median()
train['Rooms'].value_counts()

2.0    3891
1.0    3705
3.0    2235
4.0     150
5.0      18
6.0       1
Name: Rooms, dtype: int64

In [175]:
test.loc[(test['Rooms'] > 6) | (test['Rooms'] == 0)]

,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
2406,0,116.824203,113.692421,0.0,3,3,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B
2524,0,76.345154,42.820797,12.0,14,0,1977,0.017647,B,B,2,469,0,NaN,0,0,B
3398,17,52.866108,32.528343,8.0,15,17,1987,0.093443,B,B,23,4635,5,3300.0,2,4,B
